In [ ]:
! pip list

In [1]:
%env HF_ENABLE_HF_TRANSFER=1

env: HF_ENABLE_HF_TRANSFER=1


In [2]:
!pip install unsloth vllm -qU
!pip install --upgrade pillow -qU
# If you are running this notebook locally (not colab), you need to install `diffusers` too
!pip install diffusers wandb -qU

# Temporarily install a specific TRL nightly version that supports GRPO or just git dev version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 8.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 82.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 58.1 MB/s e

In [3]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF")
login(hf_token)

In [4]:
from kaggle_secrets import UserSecretsClient
from wandb import login
user_secrets = UserSecretsClient()

wb_token = user_secrets.get_secret("WB")
login(key=wb_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sp4ss (sp4ss-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
import wandb

wandb.init(
    project="Qwen2.5-3B-Instruct-reason",
    entity="sp4ss-self",
    config={
        'learning_rate': 5e-6,
        'batch_size': 1,
        'gradient_accumulation': 1,
        'adam_beta1': 0.9,
        'adam_beta2': 0.99,
        'weight_decay': 0.1,
        'warmup_ratio': 0.1,
        'lr_scheduler_type': "cosine",
        'optim': "adamw_8bit",
    },
    name="COT-qwen2.5-3b",
    notes="Experiment to tune model Qwen",
    tags=["grpo", "gsm8k", "LoRA"],
    job_type="training",
)

In [6]:
from unsloth import FastLanguageModel, PatchFastRL
# Execute the Patch
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-16 15:09:31.786033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752678571.993189      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752678572.048434      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-16 15:09:49 [__init__.py:244] Automatically detected platform cuda.


In [8]:
import torch

# Load Base Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = 2048,                      # Can increase for longer reasoning traces
    load_in_4bit = True,                        # False for LoRA 16bit
    fast_inference = True,                      # Enable vLLM fast inference
    max_lora_rank = 64,                         # Larger rank = smarter, but slower
    gpu_memory_utilization = 0.7,               # Reduce if out of memory
)

# Prepare Model for Parameter Efficient Fine Tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,                                     # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 64,                            # LoRA Rank
    use_gradient_checkpointing = "unsloth",     # Enable long context finetuning
    random_state = 3407,
)

Unsloth: vLLM does not work on older GPUs - will switch to Unsloth inference!
==((====))==  Unsloth 2025.7.3: Fast Qwen2 patching. Transformers: 4.52.4. vLLM: 0.9.2.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
SYSTEM_PROMPT = """
Please follow this structure when responding:

<reasoning>
Provide a detailed explanation of your thought process, the steps you took to arrive at the conclusion, and any relevant assumptions or considerations. This section should break down the logic behind your answer.
</reasoning>

<answer>
Present your final response in a concise and clear manner, based on the reasoning above. The answer should directly address the user's question or request.
</answer>
"""

In [ ]:
# SYSTEM_PROMPT = """
# Respond in the following format:
# <reasoning>
# ...
# </reasoning>
# <answer>
# ...
# </answer>
# """

In [10]:
import re
from datasets import load_dataset, Dataset

# Helper Function for Parsing Answer
def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()


# Helper Function to Extract Answer from LLM Response
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

# XML Counting Helper Function
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [16]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

# Defining Trainer Arguments
training_args = GRPOConfig(
    use_vllm = False,                             # Enable faster inference using vLLM
    learning_rate = 5e-6,                        # Small learning rate for stable training
    adam_beta1 = 0.9,                           # AdamW optimizer momentum parameter
    adam_beta2 = 0.99,                          # AdamW optimizer second moment parameter
    weight_decay = 0.1,                         # L2 regularization to prevent overfitting
    warmup_ratio = 0.1,                         # Portion of training steps for learning rate warmup
    lr_scheduler_type = "cosine",               # Learning rate decay schedule type
    optim = "adamw_8bit",                       # Use 8-bit AdamW optimizer for memory efficiency
    logging_steps = 1,                          # Log metrics every step
    bf16 = is_bfloat16_supported(),            # Use bfloat16 if hardware supports it
    fp16 = not is_bfloat16_supported(),        # Fallback to float16 if bfloat16 not supported
    per_device_train_batch_size = 1,           # Number of prompts per GPU
    gradient_accumulation_steps = 1,           # Number of steps to accumulate gradients
    num_generations = 8,                       # Number of responses to generate per prompt for GRPO
    max_prompt_length = 256,                   # Maximum length of input prompts in tokens
    max_completion_length = 512,               # Maximum length of model responses in tokens
    max_steps = 2000,                         # Total number of training steps
    save_steps = 250,                         # Save checkpoint every 250 steps
    max_grad_norm = 0.1,                      # Gradient clipping threshold
    report_to = "wandb",                      # Log metrics to Weights & Biases
    output_dir = "outputs",                   # Directory to save model checkpoints
)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 3 | Total steps = 2,000
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 119,734,272 of 3,205,672,960 (3.74% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
To solve this problem, let's break it down into steps:

1. **Calculate the cost for the first 10 tickets:**
   - Since the discount applies to any ticket bought that exceeds 10, the first 10 tickets are not discounted.
   - The cost for each of the first 10 tickets is $40.
   - Therefore, the total cost for the first 10 tickets is 10 * $40 = $400.

2. **Calculate the cost for the remaining 2 tickets:**
   - For the tickets bought beyond the initial 10, a 5% discount is applied.
   - The cost for each of the remaining 2 tickets after the discount is $40 * (1 - 0.05) = $40 * 0.95 = $38.
   - The total cost for the remaining 2 tickets is 2 * $38 = $76.

3. **Calculate the total amount paid:**
   - Add the cost of the first 10 tickets and the cost of the remaining 2 tickets.
  

GPUTooOldForTriton: Found Tesla P100-PCIE-16GB which is too old to be supported by the triton GPU compiler, which is used as the backend. Triton only supports devices of CUDA Capability >= 7.0, but your device is of CUDA capability 6.0

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
# Save the LoRA Adapter
model.save_lora("grpo_saved_lora")
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
user_secrets = UserSecretsClient()
hf = user_secrets.get_secret("HF")
login(hf)
# Merge to 16bit - Replace with your HF Repo, Model Name, and HF Token
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("Pingsz/Qwen2.5-3B-Instruct-reason", tokenizer, save_method = "merged_16bit", token = hf)

In [ ]:
# Test

In [ ]:
%%capture
!pip install unsloth vllm
!pip install --upgrade pillow

In [ ]:
from unsloth import FastLanguageModel
from vllm import SamplingParams
import torch

# Load the Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Pingsz/Qwen2.5-3B-Instruct-reason",
    max_seq_length = 2048,
    load_in_4bit = True,
    fast_inference = True,
    gpu_memory_utilization = 0.7,
)

In [ ]:
PROMPT = "How many r's are in the word strawberry?"

# SYSTEM_PROMPT = """
# Respond in the following format:
# <reasoning>
# ...
# </reasoning>
# <answer>
# ...
# </answer>
# """

text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : PROMPT},
], tokenize = False, add_generation_prompt = True)

In [ ]:
#Generate sequence
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
)[0].outputs[0].text

print("\n\n", output)

In [ ]:
! pip freeze > requirements.txt